<span style="font-size:32px;"><strong>Date Alignment</strong></span>


In [ ]:
import nltk
nltk.download('vader_lexicon')
import pandas as pd
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer


news_df = pd.read_csv("C:/Users/User/Desktop/10/raw_analyst_ratings.csv")
stock_df = pd.concat([
    pd.read_csv('C:/Users/User/Desktop/10/data/AAPL_historical_data.csv'),
    pd.read_csv('C:/Users/User/Desktop/10/data/AMZN_historical_data.csv'),
    pd.read_csv('C:/Users/User/Desktop/10/data/GOOG_historical_data.csv'),
    pd.read_csv('C:/Users/User/Desktop/10/data/META_historical_data.csv'),
    pd.read_csv('C:/Users/User/Desktop/10/data/MSFT_historical_data.csv'),
    pd.read_csv('C:/Users/User/Desktop/10/data/NVDA_historical_data.csv'),
    pd.read_csv('C:/Users/User/Desktop/10/data/TSLA_historical_data.csv')
], ignore_index=True)

print("News DataFrame - Data types:")
print(news_df.dtypes)
print("\nNews DataFrame - First few rows:")
print(news_df.head())

print("\nStock DataFrame - Data types:")
print(stock_df.dtypes)
print("\nStock DataFrame - First few rows:")
print(stock_df.head())

news_df['date'] = pd.to_datetime(news_df['date'], errors='coerce')
print(f"\nNews date column type: {news_df['date'].dtype}")
print(f"News date column first few values: {news_df['date'].head()}")

stock_df['Date'] = pd.to_datetime(stock_df['Date'], errors='coerce')
print(f"\nStock date column type: {stock_df['Date'].dtype}")
print(f"Stock date column first few values: {stock_df['Date'].head()}")

print(f"\nNon-datetime values in news_df['date']: {news_df[news_df['date'].isna()]}")

news_df
 

<span style="font-size:32px;"><strong>Sentiment Analysis</strong></span>

In [ ]:
sid = SentimentIntensityAnalyzer()

def get_sentiment_score(headline):
   
    sentiment = sid.polarity_scores(headline)
    return sentiment['compound']  

news_df['Sentiment_Score'] = news_df['headline'].apply(get_sentiment_score)

news_df['Sentiment_Label'] = news_df['Sentiment_Score'].apply(
    lambda x: 'positive' if x > 0.05 else ('negative' if x < -0.05 else 'neutral')
)

print("News DataFrame with Sentiment Scores and Labels:")
print(news_df[['headline', 'Sentiment_Score', 'Sentiment_Label']].head())


<span style="font-size:32px;"><strong>Daily Stock Returns</strong></span>

In [ ]:
aapl_df = pd.read_csv('C:/Users/User/Desktop/10/data/AAPL_historical_data.csv')
aapl_df['Stock'] = 'AAPL' 

amzn_df = pd.read_csv('C:/Users/User/Desktop/10/data/AMZN_historical_data.csv')
amzn_df['Stock'] = 'AMZN' 
goog_df = pd.read_csv('C:/Users/User/Desktop/10/data/GOOG_historical_data.csv')
goog_df['Stock'] = 'GOOG' 
meta_df = pd.read_csv('C:/Users/User/Desktop/10/data/META_historical_data.csv')
meta_df['Stock'] = 'META' 

msft_df = pd.read_csv('C:/Users/User/Desktop/10/data/MSFT_historical_data.csv')
msft_df['Stock'] = 'MSFT'  

nvda_df = pd.read_csv('C:/Users/User/Desktop/10/data/NVDA_historical_data.csv')
nvda_df['Stock'] = 'NVDA'  

tsla_df = pd.read_csv('C:/Users/User/Desktop/10/data/TSLA_historical_data.csv')
tsla_df['Stock'] = 'TSLA'  

stock_df = pd.concat([aapl_df, amzn_df, goog_df, meta_df, msft_df, nvda_df, tsla_df], ignore_index=True)

stock_df['Date'] = pd.to_datetime(stock_df['Date'], errors='coerce')

stock_df = stock_df.sort_values(by=['Stock', 'Date'])

stock_df['Daily_Return'] = stock_df.groupby('Stock')['Close'].pct_change() * 100

stock_df['Daily_Return'] = stock_df['Daily_Return'].fillna(0)

print("Stock DataFrame with Daily Returns:")
print(stock_df[['Date', 'Stock', 'Close', 'Daily_Return']].head(10))


<span style="font-size:32px;"><strong>Correlation Analysis</strong></span>

In [ ]:

news_df['Sentiment_Score'] = news_df['headline'].apply(get_sentiment_score)

news_df['date'] = pd.to_datetime(news_df['date'], errors='coerce')

average_daily_sentiment = news_df.groupby('date')['Sentiment_Score'].mean().reset_index()

average_daily_sentiment.columns = ['Date', 'Average_Sentiment_Score']

print("Average Daily Sentiment Scores:")
print(average_daily_sentiment.head())


In [ ]:
news_df['Sentiment_Score'] = news_df['headline'].apply(get_sentiment_score)

news_df['date'] = pd.to_datetime(news_df['date'], errors='coerce')
stock_df['Date'] = pd.to_datetime(stock_df['Date'], errors='coerce')

average_daily_sentiment = news_df.groupby('date')['Sentiment_Score'].mean().reset_index()
average_daily_sentiment.columns = ['Date', 'Average_Sentiment_Score']

stock_df['Daily_Return'] = stock_df.groupby('Stock')['Close'].pct_change() * 100
stock_df['Daily_Return'] = stock_df['Daily_Return'].fillna(0)

merged_df = pd.merge(average_daily_sentiment, stock_df, on='Date', how='inner')

correlation = merged_df['Average_Sentiment_Score'].corr(merged_df['Daily_Return'])

print("Pearson Correlation Coefficient between Average Daily Sentiment Score and Stock Daily Returns:")
print(correlation)